# Music Charts

In this example, we will extract information about music charts from tables on Wikipedia.
We will populate the Wikidata predicate "charted in" ([P2291](https://www.wikidata.org/wiki/Property:P2291)), which often occurs with the qualifiers "point in time" ([P585](https://www.wikidata.org/wiki/Property:P585)) and "ranking" ([P1352](https://www.wikidata.org/wiki/Property:P1352)).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import HTML

def preview_tables(tables, n=None):
    content = ""
    for i,table in enumerate(tables):
        df = takco.extract.htmltables.tableobj_to_dataframe(table)
        head = df.head()._repr_html_()
        content += f"<span style='align-self: flex-start; margin: 1em; '>{head}</span>"
        if n and i>n:
            break
    return HTML(f'<div style="width: 100%; overflow-x: scroll; white-space: nowrap; display:flex;">{content}</div>')

In [3]:
import takco

pred_charted_in = "http://www.wikidata.org/prop/direct/P2291"
pages = takco.wiki(pred=pred_charted_in, sample=10)
extracted = takco.TableSet.extract(source = pages)

print(f"Got {len(list(extracted))} tables")
preview_tables(extracted, 25)

Got 39 tables


In [4]:
reshaped = takco.TableSet.reshape(extracted)
print(f"Got {len(list(reshaped))} tables")
preview_tables(reshaped, 25)

Got 39 tables


In [5]:
clustered = takco.TableSet.cluster(reshaped, addcontext = ["pgTitle"], matchers=[])

print(f"Got {len(list(clustered))} tables")
preview_tables(clustered, 25)

Got 18 tables


In [6]:
linked = takco.TableSet.link(clustered, lookup_cells=False, linker_config = {}, lookup_config = {
    'class': 'SQLiteWikiLookup',
    'sqlitedb': '../../data/wdid_wpname.sqlitedb',
    'baseuri': 'http://www.wikidata.org/entity/',
    'fallback': {'class': 'MediaWikiAPI'},
})

print(f"Got {len(list(linked))} tables")
preview_tables(linked, 25)

Got 18 tables


In [18]:
searcher_config = {
    'class': "RDFSearcher",
    'encoding': "wikidata",
    'typeProperties': ["http://www.wikidata.org/prop/direct/P31"],
    'statementURIprefix': "http://www.wikidata.org/entity/statement/",
    'store': {
        'class': "HDTStore",
        'path': "/export/scratch1/home/kruit/scratch/downloads/wikidata/wikidata20200309.hdt",
    }
}
integrated = takco.TableSet.integrate(linked, pfd_threshold = 0.95, searcher_config=searcher_config)

from IPython.display import display

for table in sorted(integrated, key=lambda table: -table.get('numDataRows', 0)):
    df = takco.extract.htmltables.tableobj_to_dataframe(table)
    display( df.head() )
    print(f'Table has {len(df)} rows')
    display( table.get('properties') )
    print()

,_Page,Peak position,Variable,Chart
0,We No Speak Americano,4,2010–11,Australia ( ARIA )
1,We No Speak Americano,1,2010–11,Austria ( Ö3 Austria Top 40 )
2,We No Speak Americano,1,2010–11,Belgium ( Ultratop 50 Flanders)
3,We No Speak Americano,1,2010–11,Belgium ( Ultratop 50 Wallonia)
4,We No Speak Americano,1,2010–11,Bulgaria ( IFPI )


Table has 119 rows


{0: {3: {rdflib.term.URIRef('http://www.wikidata.org/prop/statement/P2291'): 0.10084033613445378},
  1: {rdflib.term.URIRef('http://www.wikidata.org/prop/qualifier/P1352'): 0.07563025210084033},
  2: {rdflib.term.URIRef('http://www.wikidata.org/prop/qualifier/P585'): 0.06722689075630252}}}

,_Page,Position,Variable,Chart
0,We No Speak Americano,93,2011,Canada (Canadian Hot 100)
1,We No Speak Americano,51,2011,Hungary (Dance Top 40)
2,We No Speak Americano,34,2010,Australia (ARIA)
3,We No Speak Americano,1,2010,Austria (Ö3 Austria Top 40)
4,We No Speak Americano,3,2010,Belgium (Ultratop Flanders)


Table has 48 rows


...

Table has 1 rows


{}

,_Page,Year,Award,Nominee/work,Result,Ref.
0,LOL (Basshunter album),2008,European Border Breakers Award,LOL,Won,


Table has 1 rows


{}